## original 

In [35]:
import pickle
import pandas as pd
import itertools
import copy as cp
from collections import Counter
import numpy as np
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
import os
import string
nltk.download('stopwords')
from tqdm.auto import tqdm
from datasets import load_dataset
dataset = load_dataset("poem_sentiment")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\maenz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Found cached dataset poem_sentiment (C:/Users/maenz/.cache/huggingface/datasets/poem_sentiment/default/1.0.0/4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099)


  0%|                                                                                            | 0/3 [00:00<…

In [36]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 892
    })
    validation: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 105
    })
    test: Dataset({
        features: ['id', 'verse_text', 'label'],
        num_rows: 104
    })
})

In [37]:
df=pd.DataFrame.from_dict(dataset["train"][0:700])

In [38]:
df.iloc[0:5]['verse_text']

0    with pale blue berries. in these peaceful shad...
1                  it flows so long as falls the rain,
2                   and that is why, the lonesome day,
3    when i peruse the conquered fame of heroes, an...
4              of inward strife for truth and liberty.
Name: verse_text, dtype: object

In [39]:
df_train=df

In [40]:
df_a=pd.DataFrame.from_dict(dataset["validation"])
df_b=pd.DataFrame.from_dict(dataset["test"])

In [41]:
df_test=pd.concat([df_a,df_b])

In [42]:
df_test=df_test.reset_index(drop=True)

In [43]:
df_test

,id,verse_text,label
0,0,"to water, cloudlike on the bush afar,",2
1,1,"shall yet be glad for him, and he shall bless",1
2,2,on its windy site uplifting gabled roof and pa...,2
3,3,(if haply the dark will of fate,0
4,4,"jehovah, jove, or lord!",2
...,...,...,...
204,99,shall live my highland mary.,2
205,100,now is past since last we met,2
206,101,begins to live,2
207,102,beneath the hazel bough;,2


In [44]:
# df_train.to_csv("poem_train")
# df_test.to_csv("poem_test")

In [45]:
df_train=pd.read_csv("poem_train")
df_text=pd.read_csv("poem_test")

In [46]:
X_train_original=df_train['verse_text']
X_valid_original=df_test['verse_text']
y_train_original=df_train['label']
y_valid_original=df_test['label']

In [47]:
import torch
assert torch.cuda.is_available()
device = torch.device("cuda")

In [48]:
from transformers import AutoTokenizer, AdamW,BertForSequenceClassification
tokenizer= AutoTokenizer.from_pretrained("bert-base-uncased")
X_train=tokenizer(list(X_train_original),truncation=True, padding=True, max_length=12,return_tensors='pt')
y_train=list(y_train_original)
X_valid=tokenizer(list(X_valid_original),truncation=True, padding=True, max_length=12,return_tensors='pt')
y_valid=list(y_valid_original)

In [49]:
class MyData(torch.utils.data.Dataset):
    def __init__(self, token):
        self.token = token
    def __getitem__(self, id):
        return {key: val[id] for key, val in self.token.items()}
    def __len__(self):
        return len(self.token.input_ids)

In [50]:
bat_size=128
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)
valid_dataloader = DataLoader(MyData(X_valid) , batch_size=bat_size,shuffle = False)

In [17]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 4,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [47]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)
    
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")

epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                  | 0/6 [00:00<…

validset:   0%|                                                                                  | 0/2 [00:00<…

epoch: 0
train loss: 1.302384873231252
valid loss: 1.1306560635566711
valid accuracy: 0.6602870813397129
——————————————————————————————————————————————


trainset:   0%|                                                                                  | 0/6 [00:00<…

...

validset:   0%|                                                                                  | 0/2 [00:00<…

epoch: 36
train loss: 0.05416193356116613
valid loss: 1.247413992881775
valid accuracy: 0.7320574162679426
——————————————————————————————————————————————


trainset:   0%|                                                                                  | 0/6 [00:00<…

validset:   0%|                                                                                  | 0/2 [00:00<…

early stop


In [48]:
print(max(valid_accuracy_list))

0.7464114832535885


## baseline

In [49]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase the following sentence: " +text}])
    return completion.choices[0].message.content

In [50]:
rephrase_by_chatgpt(X_train_original[1])

'It continues to flow for as long as the rain falls.'

In [51]:
# x_new=[]
# y_new=[]
# for i in tqdm(range(len(X_train_original))):
#     count=0
#     while(count!=1):
#         try:
#             ans=rephrase_by_chatgpt(X_train_original[i])
#             x_new.append(ans)
#             y_new.append(y_train_original[i])
#             count=1  
#         except:
#             time.sleep(0.5)

In [52]:
# base_dict={"verse_text":x_new,"label":y_new}
# new_df=pd.DataFrame.from_dict(base_dict)
# new_df.to_csv("chatgpt_poem_base")
new_df=pd.read_csv("chatgpt_poem_base")
X_train_base=list(X_train_original)+list(new_df['verse_text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=12,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [53]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 4,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [54]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")

epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/2 [00:00<…

epoch: 0
train loss: 1.14312021298842
valid loss: 0.9796668887138367
valid accuracy: 0.6602870813397129
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

...

validset:   0%|                                                                                  | 0/2 [00:00<…

epoch: 23
train loss: 0.06735729019750249
valid loss: 1.1525227427482605
valid accuracy: 0.7511961722488039
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/2 [00:00<…

early stop


In [55]:
print(max(valid_accuracy_list))

0.7607655502392344


### trial1

In [17]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgpt2(text,label):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    if(label==0):
        tone="negative"
    elif(label==1):
        tone="positive"
    elif(label==2):
        tone="neutral"
    else:
        tone="mixed"
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase this sentence while retaining the"+ tone + "emotion: "+text}])
    return completion.choices[0].message.content

In [18]:
rephrase_by_chatgpt2(X_train_original[1],y_train_original[1])

'It continues to flow as long as the rain falls.'

In [19]:
x_new=[]
y_new=[]
for i in tqdm(range(len(X_train_original))):
    count=0
    while(count!=1):
        try:
            ans=rephrase_by_chatgpt2(X_train_original[i],y_train_original[i])
            x_new.append(ans)
            y_new.append(y_train_original[i])
            count=1  
        except:
            time.sleep(0.5)

  0%|                                                                                          | 0/700 [00:00<…

In [20]:
base_dict={"verse_text":x_new,"label":y_new}
new_df=pd.DataFrame.from_dict(base_dict)
new_df.to_csv("chatgpt_poem_trial1")
new_df=pd.read_csv("chatgpt_poem_trial1")
X_train_base=list(X_train_original)+list(new_df['verse_text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=12,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [21]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 4,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [22]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")

epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/2 [00:00<…

epoch: 0
train loss: 1.3171073306690564
valid loss: 1.0668271780014038
valid accuracy: 0.6602870813397129
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

...

validset:   0%|                                                                                  | 0/2 [00:00<…

epoch: 15
train loss: 0.19722473350438205
valid loss: 1.0008128583431244
valid accuracy: 0.7320574162679426
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/2 [00:00<…

early stop


In [23]:
print(max(valid_accuracy_list))

0.7799043062200957


### back translation

In [29]:
import os
import openai
import time
from auto_tqdm import tqdm
def chatgpt_to_spain(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please translate this sentence to Spanish while retaining the emotion:"+text}])
    return completion.choices[0].message.content
def chatgpt_to_english(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please translate this sentence to English:"+text}])
    return completion.choices[0].message.content

In [30]:
# x_new=[]
# y_new=[]
# for i in tqdm(range(len(X_train_original))):
#     count=0
#     while(count!=1):
#         try:
#             temp=chatgpt_to_spain(X_train_original[i])
#             ans=chatgpt_to_english(temp)
#             x_new.append(ans)
#             y_new.append(y_train_original[i])
#             count=1  
#         except:
#             time.sleep(0.5)

In [31]:
# base_dict={"text":x_new,"label":y_new}
# new_df=pd.DataFrame.from_dict(base_dict)
# new_df.to_csv("poem_back_translate")
new_df=pd.read_csv("poem_back_translate")
X_train_base=list(X_train_original)+list(new_df['text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=12,return_tensors='pt')
y_train=list(y_train_base)
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)

In [32]:
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 4,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [33]:
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")

epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/2 [00:00<…

epoch: 0
train loss: 1.1549361835826526
valid loss: 1.0173993706703186
valid accuracy: 0.6602870813397129
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

...

validset:   0%|                                                                                  | 0/2 [00:00<…

epoch: 23
train loss: 0.11052980273962021
valid loss: 1.0719385147094727
valid accuracy: 0.7464114832535885
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/2 [00:00<…

early stop


In [34]:
print(max(valid_accuracy_list))

0.7655502392344498


### p2

In [18]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgptp2(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase the following sentences from poem but keep their emotion " +text}])
    return completion.choices[0].message.content
x_new=[]
y_new=[]
for i in tqdm(range(len(X_train_original))):
    count=0
    while(count!=1):
        try:
            ans=rephrase_by_chatgptp2(X_train_original[i])
            x_new.append(ans)
            y_new.append(y_train_original[i])
            count=1  
        except:
            time.sleep(0.5)
base_dict={"verse_text":x_new,"label":y_new}
new_df=pd.DataFrame.from_dict(base_dict)
new_df.to_csv("chatgpt_poem_p2")
new_df=pd.read_csv("chatgpt_poem_p2")
X_train_base=list(X_train_original)+list(new_df['verse_text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=12,return_tensors='pt')
y_train=list(y_train_base)

  0%|                                                                                          | 0/700 [00:00<…

In [19]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 4,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/2 [00:00<…

epoch: 0
train loss: 1.253892887722362
valid loss: 1.0453683137893677
valid accuracy: 0.6602870813397129
——————————————————————————————————————————————


...

validset:   0%|                                                                                  | 0/2 [00:00<…

epoch: 16
train loss: 0.4355633827773007
valid loss: 0.8328381478786469
valid accuracy: 0.7607655502392344
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/2 [00:00<…

early stop


In [20]:
print(max(valid_accuracy_list))

0.7607655502392344


## p3

In [28]:
import os
import openai
import time
from auto_tqdm import tqdm
def rephrase_by_chatgptp3(text):
    file1=open("api_key.txt",'r')
    api_key=file1.read()
    openai.api_key=api_key
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{'role': 'user', 'content': "Please rephrase the following sentences to around 12 length words without changing emotion: " + text}])
    return completion.choices[0].message.content
x_new=[]
y_new=[]
for i in tqdm(range(len(X_train_original))):
    count=0
    while(count!=1):
        try:
            ans=rephrase_by_chatgptp3(X_train_original[i])
            x_new.append(ans)
            y_new.append(y_train_original[i])
            count=1  
        except:
            time.sleep(0.5)
base_dict={"verse_text":x_new,"label":y_new}
new_df=pd.DataFrame.from_dict(base_dict)
new_df.to_csv("chatgpt_poem_p3")
new_df=pd.read_csv("chatgpt_poem_p3")
X_train_base=list(X_train_original)+list(new_df['verse_text'])
y_train_base=list(y_train_original)+list(new_df['label'])
X_train=tokenizer(list(X_train_base),truncation=True, padding=True, max_length=12,return_tensors='pt')
y_train=list(y_train_base)

  0%|                                                                                          | 0/700 [00:00<…

In [33]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(MyData(X_train) , batch_size=bat_size,shuffle = False)
model= BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 4,    
    output_attentions = False, 
    output_hidden_states = False,
)
model.to(device)
optimizer = AdamW(model.parameters(),
                  lr = 1e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8, # args.adam_epsilon  - default is 1e-8
                  weight_decay=0.01,
                  correct_bias=True
                )
epochs = 50
train_lost_epoch=[]
valid_lost_epoch=[]
valid_accuracy_list=[]
for epoch in tqdm(range(epochs),desc='epoch'):
    model.train()
    loss_list=[]
    i=0
    for batch in tqdm(train_dataloader,desc='trainset',leave=False):
        model.zero_grad() 
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        target=torch.tensor(y_train[i:i+bat_size]).to(device)
        i+=bat_size
        outputs = model(input_ids, 
                            token_type_ids, 
                            attention_mask, 
                            labels=target)

        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
    train_lost_epoch.append(np.mean(loss_list))
    model.eval()
    loss_list=[]
    i=0
    num_correct=0
    for batch in tqdm(valid_dataloader,desc='validset',leave=False):
        with torch.no_grad():  
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            target=torch.tensor(y_valid[i:i+bat_size]).to(device)
            i+=bat_size
            outputs = model(input_ids, 
                                token_type_ids, 
                                attention_mask, 
                                labels=target)
            loss = outputs.loss
            logits = outputs.logits
            loss_list.append(loss.item())
            pred_flat = np.argmax(logits.detach().cpu().numpy(), axis=1).flatten()
            labels_flat = target.to('cpu').numpy().flatten()
            num_correct += np.sum(pred_flat == labels_flat)
    valid_accuracy=num_correct/len(y_valid)
    if  epoch==0:
        torch.save(model.state_dict(), "my_model_re")
    else:
        if  valid_accuracy>max(valid_accuracy_list):
            torch.save(model.state_dict(), "my_model_re")
    if epoch>=10:
        if valid_accuracy<min(valid_accuracy_list[-10:-1]):
            print("early stop")
            break
    valid_accuracy_list.append(valid_accuracy)
    valid_lost_epoch.append(np.mean(loss_list))
    print("epoch: "+str(epoch))
    print("train loss: "+str(train_lost_epoch[-1]))
    print("valid loss: "+str(valid_lost_epoch[-1]))
    print("valid accuracy: "+str(valid_accuracy))
    print("——————————————————————————————————————————————")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

epoch:   0%|                                                                                    | 0/50 [00:00<…

trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/2 [00:00<…

epoch: 0
train loss: 1.2254488847472451
valid loss: 1.0295893549919128
valid accuracy: 0.6602870813397129
——————————————————————————————————————————————


...

validset:   0%|                                                                                  | 0/2 [00:00<…

epoch: 16
train loss: 0.2589221379973672
valid loss: 0.9660221636295319
valid accuracy: 0.7368421052631579
——————————————————————————————————————————————


trainset:   0%|                                                                                 | 0/11 [00:00<…

validset:   0%|                                                                                  | 0/2 [00:00<…

early stop


In [34]:
print(max(valid_accuracy_list))

0.7511961722488039
